In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip :
  zip.extractall()
  print('Done')

Done


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk # natural language toolkit
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
import numpy as np
import pandas as pd
import re

Data Processing

In [ ]:
#load data from csv to pandas
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
# checking the number of rows and columns
data.shape

(1599999, 6)

In [ ]:
data.sample(10)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1334209,4,2016640105,Wed Jun 03 07:07:08 PDT 2009,NO_QUERY,peretaj,Ahhh takin' a break from work it's feels so go...
531710,0,2196425294,Tue Jun 16 12:27:22 PDT 2009,NO_QUERY,NoreenDee,I'm actually thinking of marching to my neares...
1162835,4,1979622198,Sun May 31 03:50:22 PDT 2009,NO_QUERY,bwness,Got some AWESOME paint markers from mom! GREAT...
122296,0,1833682824,Mon May 18 00:05:39 PDT 2009,NO_QUERY,sarahtheissen,@Austin_Irl Have a safe trip anyways...
781469,0,2323427075,Thu Jun 25 00:37:05 PDT 2009,NO_QUERY,mikecono,Sad right now... Maybe I do have a heart and ...
646820,0,2236604690,Fri Jun 19 04:06:52 PDT 2009,NO_QUERY,felicityfuller,@Orchidflower yes it would be - she'd taken th...
1474463,4,2065732834,Sun Jun 07 09:11:00 PDT 2009,NO_QUERY,pea12nut,wants to be a professional poker player!!! and...
138000,0,1880454879,Fri May 22 01:18:09 PDT 2009,NO_QUERY,alannahwastell,still sick cant breathe through nose oh why ...
1253552,4,1996917703,Mon Jun 01 15:55:36 PDT 2009,NO_QUERY,krizzzle27,i hate finals. but im sooooo glad im getting m...
5570,0,1469123927,Tue Apr 07 05:17:48 PDT 2009,NO_QUERY,jess_roz,random thoughts pt 3 of 3: i'm onto season 3 o...


In [ ]:
# naming the columns and reading the daaset again

col_names = ['target','id','date','flag','user','text']
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1', names = col_names)

In [ ]:
data.shape

(1600000, 6)

In [ ]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting the number of missing values
data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
data = data.sample(n=50000)

In [ ]:
#checking the distribution of target column
data['target'].value_counts() # almost equal distribution

target
0    25151
4    24849
Name: count, dtype: int64

convert the target for positive tweets, 4 to 1

In [ ]:
data.replace({'target':{4:1}}, inplace = True) # in the form of a dictionary

In [ ]:
data['target'].value_counts() # no neutral tweets in the data

target
0    25151
1    24849
Name: count, dtype: int64

0 -> negative tweet

1 -> positive tweet

STEMMING


In [ ]:
# for example nothing to do with code

text = "Hello 😊! I hope you're having a great day 🌞."
text = emoji.demojize(text)

print(text)

Hello :smiling_face_with_smiling_eyes:! I hope you're having a great day :sun_with_face:.


In [ ]:
!pip install emoji # for example nothing to do with code
import emoji
p_stem = PorterStemmer()
import string

In [ ]:
dict ={}
with open('/content/slang.txt', 'r') as file:
  for line in file:
    line = line.strip()
    if line:  # Check if the line is not empty
      key, value = line.split('=', 1)  # Split on the first symbol
      dict[key] = value

print(dict)

{'AFAIK': 'As Far As I Know', 'AFK': 'Away From Keyboard', 'ASAP': 'As Soon As Possible', 'ATK': 'At The Keyboard', 'ATM': 'At The Moment', 'A3': 'Anytime, Anywhere, Anyplace', 'BAK': 'Back At Keyboard', 'BBL': 'Be Back Later', 'BBS': 'Be Back Soon', 'BFN': 'Bye For Now', 'BRB': 'Be Right Back', 'BRT': 'Be Right There', 'BTW': 'By The Way', 'B4': 'Before', 'B4N': 'Bye For Now', 'CU': 'See You', 'CUL8R': 'See You Later', 'CYA': 'See You', 'FAQ': 'Frequently Asked Questions', 'FC': 'Fingers Crossed', 'FWIW': "For What It's Worth", 'FYI': 'For Your Information', 'GAL': 'Get A Life', 'GG': 'Good Game', 'GN': 'Good Night', 'GMTA': 'Great Minds Think Alike', 'GR8': 'Great!', 'G9': 'Genius', 'IC': 'I See', 'ICQ': 'I Seek you (also a chat program)', 'ILU': ' I Love You', 'IMHO': 'In My Honest/Humble Opinion', 'IMO': 'In My Opinion', 'IOW': 'In Other Words', 'IRL': 'In Real Life', 'KISS': 'Keep It Simple, Stupid', 'LDR': 'Long Distance Relationship', 'LMAO': 'Laugh My A.. Off', 'LOL': 'Laughing

In [ ]:
def chat(text): # function to convert slangs into their full forms
  new_txt = []
  for w in text.split():
    if w.upper() in dict.keys():
      new_txt.append(dict[w.upper()])
    else:
      new_txt.append(w)
  return ' '.join(new_txt)

In [ ]:
data['text'].apply(chat)

45009      I wish I lived in America ... ... and had nice...
1406657    @loveandbooze Oh my! At least I got rid of the...
689468     @abbybradz I will miss your tweeting, and I wi...
596598     No i don*t calling with my boyfriend...i sitti...
166656     if officially done with high school.... so sad...
                                 ...                        
1373460    loving life...staying up late due to a nap I t...
685217     I finally gave up trying to txt/im this wonder...
398688     Dropped Iolaus off to get checked out. My lapt...
801993     I'm up way too early and it's rather cold. On ...
1589579    Cool first-day-of-my-weekend so far! Watched Y...
Name: text, Length: 50000, dtype: object

In [ ]:
def stemming(content):
  content = emoji.demojize(content)
  # content = content.translate(str.maketrans('','',string.punctuation))
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # if any word or letter doesnt belong to this a-z then we will remove it
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [p_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # reducing the word to its root form or stemmed form and then checking if it belongs to the list of stopwords
  # if it does not then we add it to the list
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
data['stemmed_content'] = data['text'].apply(stemming)

In [ ]:
data.sample(10)

,target,id,date,flag,user,text,stemmed_content
1138796,1,1976872182,Sat May 30 19:22:54 PDT 2009,NO_QUERY,rxgellivictor,is on air www.rx931.com from 9am to 2pm today!...,air www rx com pm today best hophop rnb hip
227909,0,1978155439,Sat May 30 22:34:50 PDT 2009,NO_QUERY,joceybrown,"I miss my boyfran, bestfran, brother, and Cuff...",miss boyfran bestfran brother cuffbff
657361,0,2240993633,Fri Jun 19 10:24:46 PDT 2009,NO_QUERY,yankeechick78,@Stefmara that sun is blinding. I miss the sun.,stefmara sun blind miss sun
170435,0,1962905897,Fri May 29 11:54:07 PDT 2009,NO_QUERY,emclev,A delay until midnight bloody thomson!!!!,delay midnight bloodi thomson
949366,1,1823712650,Sat May 16 22:45:11 PDT 2009,NO_QUERY,DeeAnA19,"Pickiin up my partner in crime,Chiqui",pickiin partner crime chiqui
382944,0,2053100151,Sat Jun 06 03:16:18 PDT 2009,NO_QUERY,zaramccartney94,"Ewww der mushed up everywere,, im never gonna ...",ewww der mush everywer im never gonna b abl ti...
620071,0,2228197575,Thu Jun 18 14:08:24 PDT 2009,NO_QUERY,gillibobs29,I am going to miss Bigbury can anything beat ...,go miss bigburi anyth beat daili walk beach
1432801,1,2060330454,Sat Jun 06 18:42:10 PDT 2009,NO_QUERY,NeoCleo_the_Cat,I went to my first adoption day today. I was s...,went first adopt day today brave girl foster m...
1110845,1,1972129700,Sat May 30 08:37:13 PDT 2009,NO_QUERY,loripluna,At starbucks. About to go get my hair dyed. Su...,starbuck go get hair dy summer blond alreadi f...
693941,0,2252863773,Sat Jun 20 07:08:34 PDT 2009,NO_QUERY,Shishaaaaa,"Tired and exhausted, helping my best friend to...",tire exhaust help best friend move apart reall...


In [ ]:
print(data['stemmed_content'])

45009                            wish live america nicer leg
1406657    loveandbooz oh least got rid horni kitti mayb ...
689468     abbybradz miss tweet miss say horribl inapprop...
596598                        call boyfriend sit school bore
166656     offici done high school sad miss alreadi movi ...
                                 ...                        
1373460    love life stay late due nap took earlier manha...
685217     final gave tri txt im wonder ladi crazi either...
398688     drop iolau get check laptop bag feel sad empti...
801993     way earli rather cold brighter side becki howa...
1589579    cool first day weekend far watch ye man seven ...
Name: stemmed_content, Length: 50000, dtype: object


In [ ]:
X = data['stemmed_content'].values
Y = data['target'].values

In [ ]:
print(X)
Y

['wish live america nicer leg'
 'loveandbooz oh least got rid horni kitti mayb twitter find peep follow sure'
 'abbybradz miss tweet miss say horribl inappropri thing irl' ...
 'drop iolau get check laptop bag feel sad empti without'
 'way earli rather cold brighter side becki howard go picnic today'
 'cool first day weekend far watch ye man seven pound underworld rise lycan fun differ reason']


array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
# Splitting into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state = 2)
# stratify to make sure equal proportion of data goes into training data and testing data i.e. 50-50
# random state so that data is split in the same way for everyone

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(50000,) (40000,) (10000,)


Vectorization

In [ ]:
vectorizer = TfidfVectorizer()
#fitting and transforming training data
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# gives importance to each word on the basis of whether it is a positive tweet or a negative tweet

In [ ]:
print(X_train)

  (0, 39849)	0.4153135183046337
  (0, 25690)	0.32466512985850454
  (0, 37355)	0.26255439025978616
  (0, 36393)	0.21576486089327096
  (0, 5034)	0.2974339528301754
  (0, 3915)	0.2795377484113281
  (0, 24732)	0.3328891418616861
  (0, 34118)	0.37119987499562673
  (0, 23994)	0.2023741350004067
  (0, 1743)	0.2641161432135873
  (0, 14891)	0.28406423085983024
  (1, 26651)	0.49408831588139057
  (1, 36640)	0.5080094974956592
  (1, 14611)	0.2546135908412706
  (1, 32121)	0.32714046687861276
  (1, 40003)	0.4548467089218962
  (1, 15380)	0.3450631713700785
  (2, 32667)	0.6412425761906152
  (2, 16342)	0.7673382295183937
  (3, 15643)	0.35381246409350386
  (3, 2197)	0.46235897146147353
  (3, 25915)	0.2322629366209899
  (3, 13644)	0.20218125656876737
  (3, 33000)	0.3857791413789623
  (3, 9695)	0.33884189840862056
  :	:
  (39996, 12284)	0.3181867508434171
  (39996, 34476)	0.35259085956516745
  (39996, 1625)	0.3381337006433575
  (39997, 8649)	0.5725311166249779
  (39997, 3809)	0.4577097730743838
  (39997, 

In [ ]:
print(X_test)

  (0, 23715)	0.7338242759038135
  (0, 14494)	0.5481184781129221
  (0, 8351)	0.40133286190569767
  (1, 38980)	0.35911760365762435
  (1, 38736)	0.3746646502662564
  (1, 37025)	0.34083647502683145
  (1, 31181)	0.4831709833628382
  (1, 14611)	0.2955791810256404
  (1, 13538)	0.19741908647626247
  (1, 8351)	0.210776178087826
  (1, 3492)	0.4585513052007594
  (2, 29729)	1.0
  (3, 20360)	0.8401622650524017
  (3, 11845)	0.30963016092572027
  (3, 11827)	0.2976120640294111
  (3, 11167)	0.33118513126540594
  (4, 39281)	0.3018592968693739
  (4, 36006)	0.2290435806336303
  (4, 35702)	0.1974696547406796
  (4, 34962)	0.32771371520101933
  (4, 34280)	0.2962255605561476
  (4, 32297)	0.23471472857164338
  (4, 24963)	0.43173277175980035
  (4, 23661)	0.2251339341733015
  (4, 13644)	0.36121156181391173
  :	:
  (9995, 13211)	0.22865882292801945
  (9995, 8351)	0.23055552464462672
  (9996, 36888)	0.33833444652522005
  (9996, 35702)	0.23572354825236574
  (9996, 34912)	0.6092142892444108
  (9996, 21263)	0.3530374

Training the ML Model

In [ ]:
model = LogisticRegression(max_iter = 1000 )
#max no of times the model goes through the data

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [ ]:
acc_score_train = model.score(X_train,Y_train)

In [ ]:
acc_score_train

0.83735

In [ ]:
acc_score_test = accuracy_score(Y_test, model.predict(X_test))

In [ ]:
acc_score_test

0.7486

In [ ]:
str = ['Its bad! @user']
x = model.predict(vectorizer.transform(str))
if x[0] == 1:
  print('Positive tweet')
else:
  print('Negative tweet')

Negative tweet


Saving the Trained Model


In [ ]:
import pickle

In [ ]:
file = 'trained.pkl'
pickle.dump(model,open(file,'wb'))

In [ ]:
load_model = pickle.load(open('/content/trained.pkl','rb'))

In [ ]:
X_new = X_test[200]